In [1]:
import re
import string
import pandas as pd
from functools import reduce
from math import log
import numpy as np

## Simple example of [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

In **information retrieval**, `tf–idf or TFIDF`, short for `term frequency–inverse document frequency`, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus, It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today; 83% of text-based recommender systems in digital libraries use tf–idf.

Variations of the tf–idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query. tf–idf can be successfully used for stop-words filtering in various subject fields, including text summarization and classification.

One of the simplest ranking functions is computed by summing the tf–idf for each query term; many more sophisticated ranking functions are variants of this simple model.





1. Example of corpus
2. Preprocessing and Tokenizing
3. Calculating bag of words
4. TF
5. IDF
6. TF-IDF

In [2]:
#1
corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
Simple example with Cats and dogs
""".split("\n")[1:-1]
corpus

['Simple example with Cats and Mouse',
 'Another simple example with dogs and cats',
 'Another simple example with mouse and cheese',
 'Simple example with Cats and dogs']

In [3]:
#2
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A,l_B,l_C)

['simple', 'example', 'with', 'cats', 'and', 'mouse'] ['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats'] ['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


In [4]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'with', 'example', 'cats', 'simple', 'and', 'dogs', 'mouse', 'another', 'cheese'}


In [5]:
len(word_set)

9

In [6]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C: 
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A,word_dict_B, word_dict_C])

,with,example,cats,simple,and,dogs,mouse,another,cheese
0,1,1,1,1,1,0,1,0,0
1,1,1,1,1,1,1,0,1,0
2,1,1,0,1,1,0,1,1,1


In [7]:
l_A

['simple', 'example', 'with', 'cats', 'and', 'mouse']

## 4 tf - Term Frequency
In the case of the term frequency $tf(t,d)$, the simplest choice is to use the raw count of a term in a string. 
$${\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}} $$
where $n_t$ is the number of occurrences of the word $t$ in the string, and in the denominator - the total number of words in this string.

**Example :** This is an Another Example 

<img src = 'https://lh3.googleusercontent.com/-4kZABcfu4uw/XjfrdNDf5OI/AAAAAAAAmgs/8i8tsCI8rsERNrm0P6WVrtaVT7I1wNvYgCK8BGAsYHg/s0/2020-02-03.png'/>

In [8]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [9]:
tf_A = compute_tf(word_dict_A, l_A)
tf_A

{'with': 0.16666666666666666,
 'example': 0.16666666666666666,
 'cats': 0.16666666666666666,
 'simple': 0.16666666666666666,
 'and': 0.16666666666666666,
 'dogs': 0.0,
 'mouse': 0.16666666666666666,
 'another': 0.0,
 'cheese': 0.0}

In [10]:
tf_B = compute_tf(word_dict_B, l_B)
tf_B

{'with': 0.14285714285714285,
 'example': 0.14285714285714285,
 'cats': 0.14285714285714285,
 'simple': 0.14285714285714285,
 'and': 0.14285714285714285,
 'dogs': 0.14285714285714285,
 'mouse': 0.0,
 'another': 0.14285714285714285,
 'cheese': 0.0}

In [11]:
tf_C = compute_tf(word_dict_C, l_C)
tf_C

{'with': 0.14285714285714285,
 'example': 0.14285714285714285,
 'cats': 0.0,
 'simple': 0.14285714285714285,
 'and': 0.14285714285714285,
 'dogs': 0.0,
 'mouse': 0.14285714285714285,
 'another': 0.14285714285714285,
 'cheese': 0.14285714285714285}

## 5 idf - Inverse Document Frequency
idf is a measure of how much information the word provides
$$ \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$
- $N$: total number of strings in the corpus ${\displaystyle N={|D|}}$
- ${\displaystyle |\{d\in D:t\in d\}|}$  : number of strings where the term ${\displaystyle t}$ appears (i.e., ${\displaystyle \mathrm {tf} (t,d)\neq 0})$. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to ${\displaystyle 1+|\{d\in D:t\in d\}|}$.

In [12]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [13]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
idf

{'with': 0.0,
 'example': 0.0,
 'cats': 0.4054651081081644,
 'simple': 0.0,
 'and': 0.0,
 'dogs': 1.0986122886681098,
 'mouse': 0.4054651081081644,
 'another': 0.4054651081081644,
 'cheese': 1.0986122886681098}

## \# 6 tf-idf Term Frequency–Inverse Document Frequency
Then tf–idf is calculated as
$$ {\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\cdot \mathrm {idf} (t,D)} $$

In [14]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [15]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [16]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,with,example,cats,simple,and,dogs,mouse,another,cheese
0,0.0,0.0,0.067578,0.0,0.0,0.000000,0.067578,0.000000,0.000000
1,0.0,0.0,0.057924,0.0,0.0,0.156945,0.000000,0.057924,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.057924,0.057924,0.156945


# For clustering we must use tf-idf weights
the example above is just an example, in practice it is better to apply [TfidfVectorizer from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Full text for clusterring

This corpus contain some strings about Google and some strings about TF-IDF from Wikipedia. Just for example

In [18]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
NQ is aimed at enabling QA systems to read and comprehend an entire Wikipedia article that may or may not contain the answer to the question. Systems will need to first decide whether the question is sufficiently well defined to be answerable — many questions make false assumptions or are just too ambiguous to be answered concisely. Then they will need to decide whether there is any part of the Wikipedia page that contains all of the information needed to infer the answer. We believe that the long answer identification task — finding all of the information required to infer an answer — requires a deeper level of language understanding than finding short answers once the long answers are known.

It is our hope that the release of NQ, and the associated challenge, will help spur the development of more effective and robust QA systems. We encourage the NLU community to participate and to help close the large gap between the performance of current state-of-the-art approaches and a human upper bound. Please visit the challenge website to view the leaderboard and learn more.
""".split("\n")[1:-1]

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

In [19]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

Now, let's calculate tf-idf for this corpus

In [20]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2',
                preprocessor=<function preprocessing at 0x000002797AB95A60>,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [21]:
tfidf = tfidf_vectorizer.fit_transform(all_text)
tfidf

<16x187 sparse matrix of type '<class 'numpy.float64'>'
	with 258 stored elements in Compressed Sparse Row format>

In [22]:
tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.30193189, 0.30193189, 0.        , ..., 0.        , 0.        ,
        0.26294454],
       [0.        , 0.        , 0.        , ..., 0.35843178, 0.        ,
        0.31214881],
       ...,
       [0.        , 0.        , 0.07360375, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [23]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.30193189, 0.30193189, 0.        , ..., 0.        , 0.        ,
         0.26294454],
        [0.        , 0.        , 0.        , ..., 0.35843178, 0.        ,
         0.31214881],
        ...,
        [0.        , 0.        , 0.07360375, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

#### CountVectorizer

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
count_vect = CountVectorizer(stop_words='english')

In [26]:
sf = count_vect.fit_transform(all_text)
sf

<16x141 sparse matrix of type '<class 'numpy.int64'>'
	with 163 stored elements in Compressed Sparse Row format>

In [27]:
sf.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
sf.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 2, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#### TfidfTransformer

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

In [30]:
trans = TfidfTransformer()

In [31]:
trans_weight = trans.fit_transform(sf)
trans_weight

<16x141 sparse matrix of type '<class 'numpy.float64'>'
	with 163 stored elements in Compressed Sparse Row format>

In [32]:
weights = np.asarray(trans_weight.mean(axis=0).ravel().tolist())
weights.shape

(1, 141)

In [33]:
weight_df = pd.DataFrame({'Term':count_vect.get_feature_names(),
                         'Weights':weights.ravel()})
weight_df

,Term,Weights
0,18,0.021851
1,60,0.021851
2,aimed,0.006711
3,ambiguous,0.006711
4,android,0.057262
...,...,...
136,visit,0.010237
137,website,0.010237
138,weighting,0.026982
139,wikipedia,0.013422


In [34]:
weight_df.sort_values(by='Weights',ascending=True).head(10)

,Term,Weights
75,make,0.006711
54,identification,0.006711
129,understanding,0.006711
23,comprehend,0.006711
24,concisely,0.006711
25,contain,0.006711
26,contains,0.006711
38,enabling,0.006711
88,page,0.006711
31,deeper,0.006711


And train simple kmeans model with k = 2

In [36]:
kmeans = KMeans(n_clusters=2).fit(tfidf)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

### Predictions

In [37]:
lines_for_predicting = ["tf and idf is awesome!", "some androids is there"]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([1, 0])

In [38]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0])

In [39]:
kmeans.n_clusters

2

In [59]:
kmeans.score(tfidf)

-11.544302971981818